# * VINSIGHT : Data Monitoring

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_monthly_file = f'vis_monthly_{str_curr_dt}'
op_daily_file = f'vis_daily_{str_curr_dt}'
op_ccaa_ccaatt_file = f'vis_ccaa_ccaatt_{str_curr_dt}'

v_year = 0
v_month_start = 0
v_month_end = 0
v_date = 20240101

# print(f"\nParameter input...\n\n   -> op_dir: {op_dir}\n   -> op_monthly_file: {op_monthly_file}\n   -> op_daily_file: {op_daily_file}")
print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_monthly_file: {op_monthly_file}')
print(f'   -> op_daily_file: {op_daily_file}')
print(f'   -> op_ccaa_ccaatt_file: {op_ccaa_ccaatt_file}')
print(f'\n   -> v_year: {v_year}')
print(f'   -> v_month_start: {v_month_start}')
print(f'   -> v_month_end: {v_month_end}')
print(f'   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_monthly_file: vis_monthly_20251224
   -> op_daily_file: vis_daily_20251224
   -> op_ccaa_ccaatt_file: vis_ccaa_ccaatt_20251224

   -> v_year: 0
   -> v_month_start: 0
   -> v_month_end: 0
   -> v_date: 20240101


## Aggregate : Timestamp

In [3]:
''' Execute : AGG_PERF_NEWCO '''


# Input parameter
v_start_date = 20251201
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ PPN_TM, AREA_TYPE
        , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT
        , MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE CENTER IN ('Revenue', 'Sales', 'Profitability')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY PPN_TM, AREA_TYPE
    ORDER BY PPN_TM, AREA_TYPE
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    vis_timestamp_df = chk_src_df.copy()
    print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20251201

Data as of 2025-12-24, 15:19:21



Current Timestamp...

                PPN_TM AREA_TYPE  CNT_METRIC  ROW_CNT    MIN_DT    MAX_DT  LAST_ACTUAL
0  2025-12-24 02:40:05         C         283     6509  20251201  20251223     20251222
1  2025-12-24 02:40:05         G         446    82064  20251201  20251223     20251222
2  2025-12-24 02:40:05         P         569    13087  20251201  20251223     20251222
3  2025-12-24 02:40:05         Z          29     4669  20251201  20251223     20251222
4  2025-12-24 02:40:33         C           4       92  20251201  20251223     20251223
5  2025-12-24 02:40:33         G           4      736  20251201  20251223     20251223
6  2025-12-24 02:40:33         P           4       92  20251201  20251223     20251223
7  2025-12-24 04:17:11         H         445   655040  20251201  20251223     20251222
8  2025-12-24 04:21:24         H           4     5888  20251201  20251223     20251223
9  2025-12-24 06:50:04        HH         399   880992  20251201  20251223     20251222
10 2025-12-24 06:55:

In [4]:
''' Execute : AGG_PERF_NEWCO_CCAA '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20251201
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ AREA_TYPE
        , MAX(PPN_TM) PPN_TM
        , COUNT(1) ROW_CNT, MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
        , COUNT(DISTINCT METRIC_CD) CNT_METRIC
    FROM GEOSPCAPPO.AGG_PERF_NEWCO_CCAA 
    WHERE TM_KEY_DAY >= {v_start_date} 
    GROUP BY AREA_TYPE
    ORDER BY 1
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    vis_timestamp_df = chk_src_df.copy()
    print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-12-24, 15:21:10
   -> v_start_date: 20251201



Current Timestamp...

  AREA_TYPE              PPN_TM  ROW_CNT    MIN_DT    MAX_DT  LAST_ACTUAL  CNT_METRIC
0      CCAA 2025-12-24 09:33:14  5946624  20251201  20251224     20251222         267


In [5]:
# ''' Execute : AGG_PERF_NEWCO_CCAATT '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20251101
# print(f'\nParameter input...\n')
# print(f'   -> Data as of {curr_datetime}')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ AREA_TYPE
#         , MAX(PPN_TM) PPN_TM
#         , COUNT(1) ROW_CNT, MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
#         , COUNT(DISTINCT METRIC_CD) CNT_METRIC
#     FROM GEOSPCAPPO.AGG_PERF_NEWCO_CCAATT 
#     WHERE TM_KEY_DAY >= {v_start_date} 
#     GROUP BY AREA_TYPE
#     ORDER BY 1
# """)


# try:
#     # Get : Timestamp
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     print(f'\nCurrent Timestamp...')
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

#     # Display
#     vis_timestamp_df = chk_src_df.copy()
#     print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()